# Sistema RAG Completo - Llama 3.1 8B

Este notebook combina a construção do índice e a execução do sistema RAG com avaliação de métricas.

## Estrutura:
1. **Configuração e Imports**
2. **Construção do Índice** (executar 1x e salvar)
3. **Carregamento do Sistema RAG** (carregar modelos salvos)
4. **Execução de Perguntas** (rodar N vezes)
5. **Avaliação de Métricas** (análise separada dos resultados)

---
## 1. Configuração e Imports

In [ ]:
%%writefile requirements.txt

# --- Índices para pacotes pré-compilados com CUDA 12.4 ---
--extra-index-url https://download.pytorch.org/whl/cu126
--extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu124

# --- Versões Fixas para Estabilidade ---

# Core de IA e Machine Learning
torch
torchvision
torchaudio
#numpy
#pandas

# Bibliotecas do RAG
langchain
langchain-community
langchain-core
sentence-transformers
faiss-gpu-cu12
llama-cpp-python

# Ferramentas e Utilitários
PyMuPDF
unstructured[pdf]==0.18.15
huggingface_hub
evaluate
rouge_score
tqdm
poppler-utils
bert_score



Writing requirements.txt


In [1]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu126, https://abetlen.github.io/llama-cpp-python/whl/cu124


In [2]:
import os
import shutil
import torch
import pickle
import re
import gc
import pandas as pd
import pymupdf
from tqdm import tqdm
from langchain_core.documents import Document
from langchain.storage import InMemoryStore
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.retrievers import ParentDocumentRetriever
from langchain.prompts import PromptTemplate
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.llms import LlamaCpp
from langchain.chains import LLMChain
from huggingface_hub import hf_hub_download
from sentence_transformers import CrossEncoder
import evaluate

print("✓ Imports realizados com sucesso!")
print(f"Device disponível: {'CUDA' if torch.cuda.is_available() else 'CPU'}")

✓ Imports realizados com sucesso!
Device disponível: CUDA


In [3]:
# PARÂMETROS GLOBAIS
DOCS_PATH = "documentos/"
VECTORSTORE_PATH = "faiss_final_index"
DOCSTORE_PATH = "final_docstore"
MODEL_EMBEDDING_NAME = "BAAI/bge-m3"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Configuração do Modelo Llama 3.1
MODEL_REPO_ID = "lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF"
MODEL_BASENAME = "Meta-Llama-3.1-8B-Instruct-Q5_K_M.gguf"
LLM_N_GPU_LAYERS = -1
LLM_N_CTX = 8192
RETRIEVER_K = 10
TOP_K_AFTER_RERANK = 4
CROSS_ENCODER_MODEL = 'cross-encoder/ms-marco-MiniLM-L-6-v2'

print("✓ Parâmetros configurados!")

✓ Parâmetros configurados!


---
## 2. Construção do Índice FAISS
**⚠️ Execute esta seção apenas UMA VEZ ou quando atualizar os documentos**

In [4]:
def cleanup_old_index(vectorstore_path, docstore_path):
    """Limpa os diretórios antigos do índice e do docstore."""
    for path in [vectorstore_path, docstore_path]:
        if os.path.exists(path):
            shutil.rmtree(path)
    os.makedirs(vectorstore_path, exist_ok=True)
    os.makedirs(docstore_path, exist_ok=True)
    print("Limpeza e recriação das pastas concluída.")

def load_and_clean_definitively(folder_path: str) -> list:
    """
    Solução definitiva: Extrai blocos de texto, ignora cabeçalhos/rodapés,
    ordena pela ordem de leitura, une o conteúdo e normaliza os espaços.
    """
    pdf_files = [f for f in os.listdir(folder_path) if f.endswith(".pdf")]
    all_cleaned_pages = []

    for filename in tqdm(pdf_files, desc="Processando PDFs de forma definitiva"):
        file_path = os.path.join(folder_path, filename)

        with pymupdf.open(file_path) as doc:
            for page_num, page in enumerate(doc):
                page_height = page.rect.height
                margin_top = page_height * 0.08
                margin_bottom = page_height * 0.92

                blocks = page.get_text("blocks")
                valid_blocks = [b for b in blocks if margin_top < b[1] < margin_bottom]
                valid_blocks.sort(key=lambda b: b[1])

                page_text = " ".join([b[4].replace('\n', ' ') for b in valid_blocks])
                all_cleaned_pages.append(page_text)

    full_text_content = " ".join(all_cleaned_pages)
    full_text_content = re.sub(r'\s+', ' ', full_text_content).strip()

    print(f"Limpeza concluída. Conteúdo total com {len(full_text_content)} caracteres.")
    return [Document(page_content=full_text_content)]

print("✓ Funções de processamento definidas!")

✓ Funções de processamento definidas!


In [5]:
# EXECUTAR APENAS QUANDO NECESSÁRIO RECONSTRUIR O ÍNDICE
print("Iniciando construção do índice...\n")

cleanup_old_index(VECTORSTORE_PATH, DOCSTORE_PATH)
docs = load_and_clean_definitively(DOCS_PATH)

# Splitters para a estratégia Parent/Child
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200, add_start_index=True)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)

embeddings = HuggingFaceEmbeddings(model_name=MODEL_EMBEDDING_NAME, model_kwargs={'device': DEVICE})

vectorstore = FAISS.from_texts(texts=["_INITIALIZING_"], embedding=embeddings)
vectorstore.delete(list(vectorstore.index_to_docstore_id.values()))

store = InMemoryStore()

retriever = ParentDocumentRetriever(
    vectorstore=vectorstore, docstore=store,
    child_splitter=child_splitter, parent_splitter=parent_splitter,
)

print("Adicionando documento limpo ao índice...")
retriever.add_documents(docs, ids=None)

vectorstore.save_local(VECTORSTORE_PATH)
with open(os.path.join(DOCSTORE_PATH, "store.pkl"), "wb") as f:
    pickle.dump(store, f)

print(f"\n✓ Índice definitivo salvo em '{VECTORSTORE_PATH}' e '{DOCSTORE_PATH}'")

Iniciando construção do índice...

Limpeza e recriação das pastas concluída.


Processando PDFs de forma definitiva: 100%|██████████| 1/1 [00:00<00:00,  1.84it/s]
/tmp/ipython-input-712658109.py:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=MODEL_EMBEDDING_NAME, model_kwargs={'device': DEVICE})


Limpeza concluída. Conteúdo total com 232415 caracteres.


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Adicionando documento limpo ao índice...

✓ Índice definitivo salvo em 'faiss_final_index' e 'final_docstore'


---
## 3. Carregamento do Sistema RAG
**Execute esta seção para carregar os índices e modelos já salvos**

In [8]:
# Carregar embeddings
print("Carregando modelo de embeddings...")
embeddings = HuggingFaceEmbeddings(
    model_name=MODEL_EMBEDDING_NAME,
    model_kwargs={'device': DEVICE}
)
print("✓ Embeddings carregados!")

Carregando modelo de embeddings...
✓ Embeddings carregados!


In [9]:
# Carregar vectorstore FAISS
print("Carregando índice FAISS...")
vectorstore = FAISS.load_local(
    VECTORSTORE_PATH,
    embeddings,
    allow_dangerous_deserialization=True
)
print("✓ Vectorstore carregado!")

Carregando índice FAISS...
✓ Vectorstore carregado!


In [10]:
# Carregar docstore
print("Carregando docstore...")
with open(os.path.join(DOCSTORE_PATH, "store.pkl"), "rb") as f:
    store = pickle.load(f)

child_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter
)
retriever.search_kwargs['k'] = RETRIEVER_K
print("✓ Retriever configurado!")

Carregando docstore...
✓ Retriever configurado!


In [11]:
# Carregar cross-encoder
print("Carregando cross-encoder...")
cross_encoder = CrossEncoder(CROSS_ENCODER_MODEL)
print("✓ Cross-encoder carregado!")

Carregando cross-encoder...


config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

✓ Cross-encoder carregado!


In [12]:
# Baixar e carregar LLM
print(f"Baixando modelo: {MODEL_BASENAME}...")
model_path = hf_hub_download(repo_id=MODEL_REPO_ID, filename=MODEL_BASENAME)
print(f"✓ Modelo baixado: {model_path}")

print("\nInicializando LLM...")
llm = LlamaCpp(
    model_path=model_path,
    n_gpu_layers=LLM_N_GPU_LAYERS,
    n_ctx=LLM_N_CTX,
    max_tokens=512,
    repeat_penalty=1.15,
    temperature=0.1,
    top_p=0.9,
    top_k=40,
    n_batch=512,
    f16_kv=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    verbose=False
)
print("✓ LLM carregado!")

Baixando modelo: Meta-Llama-3.1-8B-Instruct-Q5_K_M.gguf...


Meta-Llama-3.1-8B-Instruct-Q5_K_M.gguf:   0%|          | 0.00/5.73G [00:00<?, ?B/s]

✓ Modelo baixado: /root/.cache/huggingface/hub/models--lmstudio-community--Meta-Llama-3.1-8B-Instruct-GGUF/snapshots/8601e6db71269a2b12255ebdf09ab75becf22cc8/Meta-Llama-3.1-8B-Instruct-Q5_K_M.gguf

Inicializando LLM...


llama_context: n_ctx_per_seq (8192) < n_ctx_train (131072) -- the full capacity of the model will not be utilized


✓ LLM carregado!


In [13]:
# Criar chain RAG
template = (
    "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n"
    "Você é um assistente especializado em responder perguntas com base em documentos acadêmicos.\n\n"
    "INSTRUÇÕES:\n"
    "- Leia atentamente o contexto fornecido\n"
    "- Responda de forma completa e natural, como em uma conversa\n"
    "- Use APENAS informações presentes no contexto\n"
    "- Para perguntas sobre 'quando', forneça a resposta em uma frase completa mencionando o ano e o local\n"
    "- Seja preciso mas não robotizado\n"
    "- Evite respostas de uma única palavra ou número<|eot_id|>"
    "<|start_header_id|>user<|end_header_id|>\n\n"
    "Contexto:\n{context}\n\n"
    "Pergunta: {question}<|eot_id|>"
    "<|start_header_id|>assistant<|end_header_id|>\n\n"
)

prompt = PromptTemplate(template=template, input_variables=["context", "question"])
rag_chain = LLMChain(prompt=prompt, llm=llm)

print("✓ RAG Chain criada!")
print("\n" + "="*80)
print("SISTEMA RAG PRONTO PARA USO!")
print("="*80)

✓ RAG Chain criada!

SISTEMA RAG PRONTO PARA USO!


/tmp/ipython-input-927304356.py:19: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  rag_chain = LLMChain(prompt=prompt, llm=llm)


---
## 4. Funções de Execução de Perguntas

In [14]:
def rerank_documents(question, retrieved_docs, cross_encoder, top_k):
    """Reordena os documentos usando um CrossEncoder."""
    pairs = [[question, doc.page_content] for doc in retrieved_docs]
    scores = cross_encoder.predict(pairs)
    doc_scores = list(zip(retrieved_docs, scores))
    doc_scores_sorted = sorted(doc_scores, key=lambda x: x[1], reverse=True)

    print(f"\n[DEBUG] Top {top_k} documentos após re-ranking:")
    for i, (doc, score) in enumerate(doc_scores_sorted[:top_k], 1):
        preview = doc.page_content[:80].replace('\n', ' ')
        print(f"  #{i} (score: {score:.4f}): {preview}...")

    return [doc for doc, score in doc_scores_sorted[:top_k]]

def ask(question, rag_chain, retriever, cross_encoder, verbose=True):
    """Pipeline completo: Retrieve -> Re-rank -> Generate."""
    if verbose:
        print(f"\n{'='*80}")
        print(f"PERGUNTA: {question}")
        print('='*80)

    retrieved_docs = retriever.invoke(question)
    if verbose:
        print(f"✓ Recuperados {len(retrieved_docs)} documentos")

    reranked_docs = rerank_documents(
        question, retrieved_docs, cross_encoder, TOP_K_AFTER_RERANK)

    context_parts = []
    for i, doc in enumerate(reranked_docs, 1):
        context_parts.append(f"[Trecho {i}]: {doc.page_content}")

    context = "\n\n".join(context_parts)

    if verbose:
        print(f"\n[DEBUG] Primeiros 400 caracteres do contexto:")
        print("-" * 80)
        print(context[:400])
        print("-" * 80)

    result = rag_chain.invoke({"context": context, "question": question})
    answer = result['text'].strip()

    # Limpeza de tokens especiais do Llama 3.1
    tokens_to_remove = ["</s>", "<|eot_id|>", "<|end_header_id|>", "<|start_header_id|>"]
    for token in tokens_to_remove:
        answer = answer.replace(token, "")

    answer = answer.strip()

    if verbose:
        print(f"\n{'='*80}")
        print(f"RESPOSTA: {answer}")
        print('='*80)

    return answer, context

print("✓ Funções de execução definidas!")

✓ Funções de execução definidas!


---
## 5. Dataset de Avaliação
**Adicione ou modifique as perguntas aqui**

In [47]:
# Dataset de perguntas e respostas esperadas
evaluation_dataset = [
    {
        "pergunta": "Quando o curso de Tecnologia em Análise e Desenvolvimento de Sistemas foi implantado no Campus de Picos?",
        "resposta_esperada": "O curso foi implantado no Campus de Picos em 2013."
    },
    {
        "pergunta": "Quais são as modalidades aceitas para a apresentação do TCC?",
        "resposta_esperada": "O TCC pode ser apresentado em formato de Monografia, Artigo Científico, Relatório Técnico de Software (RTS) ou Relatório Técnico de Trabalho/Estágio (RTT)."
    },
    {
        "pergunta": "Qual foi o montante total de investimento que o setor de TI no Brasil atingiu em 2021, englobando os mercados de software, serviços, hardware e exportações?",
        "resposta_esperada": "O investimento atingiu R$238,2 bilhões (US$ 46,2 bilhões)."
    },
    {
        "pergunta": "Qual o pré-requisito para cursar Programação para Dispositivos Móveis?",
        "resposta_esperada": "O pré-requisito para cursar Programação para Dispositivos Móveis é a disciplina de Programação Orientada a Objetos."
    },

]

print(f"✓ Dataset com {len(evaluation_dataset)} perguntas carregado!")

✓ Dataset com 4 perguntas carregado!


---
## 6. Execução das Perguntas
**Execute esta célula para processar todas as perguntas do dataset**

In [48]:
# Armazenar resultados
results = []

for idx, item in enumerate(evaluation_dataset, 1):
    print(f"\n\n{'#'*80}")
    print(f"PROCESSANDO PERGUNTA {idx}/{len(evaluation_dataset)}")
    print(f"{'#'*80}")

    generated_answer, retrieved_context = ask(
        item["pergunta"],
        rag_chain,
        retriever,
        cross_encoder,
        verbose=True
    )

    results.append({
        "pergunta": item["pergunta"],
        "resposta_esperada": item["resposta_esperada"],
        "resposta_gerada": generated_answer,
        "contexto_recuperado": retrieved_context
    })

# Criar DataFrame com os resultados
df_results = pd.DataFrame(results)

print("\n\n" + "="*80)
print("✓ TODAS AS PERGUNTAS PROCESSADAS!")
print("="*80)
print(f"Total de perguntas: {len(results)}")



################################################################################
PROCESSANDO PERGUNTA 1/4
################################################################################

PERGUNTA: Quando o curso de Tecnologia em Análise e Desenvolvimento de Sistemas foi implantado no Campus de Picos?
✓ Recuperados 8 documentos

[DEBUG] Top 4 documentos após re-ranking:
  #1 (score: 7.1133): denominação: Análise e Desenvolvimento de Sistemas. Em 2002, foi autorizada a cr...
  #2 (score: 4.9391): O Instituto Federal do Piauí possui atualmente 20 campi distribuídos do norte ao...
  #3 (score: 4.6784): MINISTÉRIO DA EDUCAÇÃO Secretaria de Educação Profissional e Tecnológica Institu...
  #4 (score: 3.6548): de Redação do Vestibular/Exame Nacional do Ensino Médio (ENEM) em um dos últimos...

[DEBUG] Primeiros 400 caracteres do contexto:
--------------------------------------------------------------------------------
[Trecho 1]: denominação: Análise e Desenvolvimento de Sistemas. Em 2002, 

/usr/local/lib/python3.12/dist-packages/llama_cpp/llama.py:1242: RuntimeWarning: Detected duplicate leading "<|begin_of_text|>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


O curso de Tecnologia em Análise e Desenvolvimento de Sistemas foi implantado no Campus de Picos, em 2013.
RESPOSTA: O curso de Tecnologia em Análise e Desenvolvimento de Sistemas foi implantado no Campus de Picos, em 2013.


################################################################################
PROCESSANDO PERGUNTA 2/4
################################################################################

PERGUNTA: Quais são as modalidades aceitas para a apresentação do TCC?
✓ Recuperados 6 documentos

[DEBUG] Top 4 documentos após re-ranking:
  #1 (score: 4.3135): entre o primeiro e o último período do curso; - Trabalho de Conclusão de Curso (...
  #2 (score: 4.0929): de Projeto Carga horária: 40h/a Módulo: IV Código(s) do Pré-requisito 17 a 23 Co...
  #3 (score: 4.0589): estará sujeito a uma arguição aberta aos membros da banca examinadora e à platei...
  #4 (score: 3.2253): de elaboração de trabalhos acadêmicos do IFPI. Os artigos científicos completos ...

[DEBUG] Primeiros 40

/usr/local/lib/python3.12/dist-packages/llama_cpp/llama.py:1242: RuntimeWarning: Detected duplicate leading "<|begin_of_text|>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


De acordo com o texto, as modalidades aceitas para a apresentação do TCC são:

1. Monografia;
2. Artigo Científico completo (com resultados) publicado em eventos regionais, nacionais e internacionais ou em periódicos pertencentes à Lista de Periódicos classificados no Qualis Capes ou na Revista Somma do IFPI.

Além disso, os alunos também podem desenvolver um Relatório Técnico de Software (RTS) ou um Relatório Técnico de Trabalho/Estágio (RTT).
RESPOSTA: De acordo com o texto, as modalidades aceitas para a apresentação do TCC são:

1. Monografia;
2. Artigo Científico completo (com resultados) publicado em eventos regionais, nacionais e internacionais ou em periódicos pertencentes à Lista de Periódicos classificados no Qualis Capes ou na Revista Somma do IFPI.

Além disso, os alunos também podem desenvolver um Relatório Técnico de Software (RTS) ou um Relatório Técnico de Trabalho/Estágio (RTT).


################################################################################
PROCESSAN

/usr/local/lib/python3.12/dist-packages/llama_cpp/llama.py:1242: RuntimeWarning: Detected duplicate leading "<|begin_of_text|>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


Em 2021, o setor de TI no Brasil atingiu um investimento total de R$238,2 bilhões (US$ 46,2 bilhões), englobando os mercados de software, serviços, hardware e exportações.
RESPOSTA: Em 2021, o setor de TI no Brasil atingiu um investimento total de R$238,2 bilhões (US$ 46,2 bilhões), englobando os mercados de software, serviços, hardware e exportações.


################################################################################
PROCESSANDO PERGUNTA 4/4
################################################################################

PERGUNTA: Qual o pré-requisito para cursar Programação para Dispositivos Móveis?
✓ Recuperados 10 documentos

[DEBUG] Top 4 documentos após re-ranking:
  #1 (score: 2.3419): complementar: 1. WASLAWICK, Raul Sidnei. Introdução a algoritmos e programação c...
  #2 (score: -2.4429): Pré-requisito: Não possui Competência e Habilidades: Apresentação de temas atuai...
  #3 (score: -3.9853): desenvolvimento para a Internet no desenvolvimento de softwares Dinâ

/usr/local/lib/python3.12/dist-packages/llama_cpp/llama.py:1242: RuntimeWarning: Detected duplicate leading "<|begin_of_text|>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


O pré-requisito para cursar Programação para Dispositivos Móveis é a disciplina "Programação Orientada a Objetos" (Competência e Habilidades: Compreender conhecimentos teóricos e práticos, acerca do domínio da programação para dispositivos móveis).
RESPOSTA: O pré-requisito para cursar Programação para Dispositivos Móveis é a disciplina "Programação Orientada a Objetos" (Competência e Habilidades: Compreender conhecimentos teóricos e práticos, acerca do domínio da programação para dispositivos móveis).


✓ TODAS AS PERGUNTAS PROCESSADAS!
Total de perguntas: 4


In [49]:
# Visualizar resultados
print("\nPRÉVIA DOS RESULTADOS:\n")
for idx, row in df_results.iterrows():
    print(f"\n{'='*80}")
    print(f"PERGUNTA {idx+1}:")
    print(f"  {row['pergunta']}")
    print(f"\nESPERADA:")
    print(f"  {row['resposta_esperada']}")
    print(f"\nGERADA:")
    print(f"  {row['resposta_gerada']}")
    print("="*80)


PRÉVIA DOS RESULTADOS:


PERGUNTA 1:
  Quando o curso de Tecnologia em Análise e Desenvolvimento de Sistemas foi implantado no Campus de Picos?

ESPERADA:
  O curso foi implantado no Campus de Picos em 2013.

GERADA:
  O curso de Tecnologia em Análise e Desenvolvimento de Sistemas foi implantado no Campus de Picos, em 2013.

PERGUNTA 2:
  Quais são as modalidades aceitas para a apresentação do TCC?

ESPERADA:
  O TCC pode ser apresentado em formato de Monografia, Artigo Científico, Relatório Técnico de Software (RTS) ou Relatório Técnico de Trabalho/Estágio (RTT).

GERADA:
  De acordo com o texto, as modalidades aceitas para a apresentação do TCC são:

1. Monografia;
2. Artigo Científico completo (com resultados) publicado em eventos regionais, nacionais e internacionais ou em periódicos pertencentes à Lista de Periódicos classificados no Qualis Capes ou na Revista Somma do IFPI.

Além disso, os alunos também podem desenvolver um Relatório Técnico de Software (RTS) ou um Relatório Téc

---
## 7. Avaliação de Métricas
**Execute esta seção após processar as perguntas para calcular as métricas**

In [50]:
# Preparar dados para métricas
generated_answers = df_results["resposta_gerada"].tolist()
expected_answers = df_results["resposta_esperada"].tolist()

print(f"Preparando avaliação de {len(generated_answers)} respostas...")

Preparando avaliação de 4 respostas...


In [51]:
# Carregar métricas
print("Carregando métricas de avaliação...\n")

rouge_metric = evaluate.load('rouge')
bleu_metric = evaluate.load('bleu')
bertscore_metric = evaluate.load('bertscore')

print("✓ Métricas carregadas!")

Carregando métricas de avaliação...

✓ Métricas carregadas!


In [20]:
# Calcular ROUGE
print("Calculando ROUGE...")
rouge_scores = rouge_metric.compute(
    predictions=generated_answers,
    references=expected_answers
)
print("✓ ROUGE calculado!")

Calculando ROUGE...
✓ ROUGE calculado!


In [52]:
# Calcular BLEU
print("Calculando BLEU...")
bleu_scores = bleu_metric.compute(
    predictions=generated_answers,
    references=expected_answers
)
print("✓ BLEU calculado!")

Calculando BLEU...
✓ BLEU calculado!


In [53]:
# Calcular BERTScore (pode demorar)
print("Calculando BERTScore... (pode demorar)")
bertscore_scores = bertscore_metric.compute(
    predictions=generated_answers,
    references=expected_answers,
    lang='pt',
    model_type='distilbert-base-multilingual-cased'
)
print("✓ BERTScore calculado!")

Calculando BERTScore... (pode demorar)
✓ BERTScore calculado!


In [54]:
# Calcular F1-Score
def calculate_f1_token(prediction, reference):
    pred_tokens = set(prediction.lower().split())
    ref_tokens = set(reference.lower().split())

    if len(pred_tokens) == 0 or len(ref_tokens) == 0:
        return 0.0

    common = pred_tokens.intersection(ref_tokens)
    precision = len(common) / len(pred_tokens)
    recall = len(common) / len(ref_tokens)

    if precision + recall == 0:
        return 0.0

    return 2 * (precision * recall) / (precision + recall)

print("Calculando F1-Score...")
f1_scores = [calculate_f1_token(p, r) for p, r in zip(generated_answers, expected_answers)]

avg_f1 = sum(f1_scores) / len(f1_scores)
avg_bertscore = sum(bertscore_scores['f1']) / len(bertscore_scores['f1'])

print("✓ F1-Scorecalculado!")

# Adicionar métricas ao DataFrame
df_results['f1_score'] = f1_scores

Calculando F1-Score...
✓ F1-Scorecalculado!


### 7.1 Visualização dos Resultados das Métricas

In [55]:
# Exibir resultados completos
print("\n" + "="*80)
print("RESULTADOS COMPLETOS DAS MÉTRICAS")
print("="*80)

print("\n[ROUGE - Overlap de N-gramas]")
print(f"  ROUGE-1: {rouge_scores['rouge1']:.4f}")
print(f"  ROUGE-2: {rouge_scores['rouge2']:.4f}")
print(f"  ROUGE-L: {rouge_scores['rougeL']:.4f}")

print("\n[BLEU - Precisão de N-gramas]")
print(f"  BLEU Score: {bleu_scores['bleu']:.4f}")

print("\n[BERTScore - Similaridade Semântica]")
print(f"  F1 (média): {avg_bertscore:.4f}")
print(f"  Precision:  {sum(bertscore_scores['precision'])/len(bertscore_scores['precision']):.4f}")
print(f"  Recall:     {sum(bertscore_scores['recall'])/len(bertscore_scores['recall']):.4f}")

print("\n[F1-Score Token-based]")
print(f"  F1 (média): {avg_f1:.4f}")

print("\n" + "="*80)


RESULTADOS COMPLETOS DAS MÉTRICAS

[ROUGE - Overlap de N-gramas]
  ROUGE-1: 0.6416
  ROUGE-2: 0.4720
  ROUGE-L: 0.5351

[BLEU - Precisão de N-gramas]
  BLEU Score: 0.2528

[BERTScore - Similaridade Semântica]
  F1 (média): 0.9223
  Precision:  0.8792
  Recall:     0.9699

[F1-Score Token-based]
  F1 (média): 0.5119



In [56]:
# Exibir métricas por pergunta e métricas gerais em tabela
print("\nMÉTRICAS DETALHADAS E POR PERGUNTA:\n")

# Criar DataFrame com as métricas gerais
metrics_data = {
    'Métrica': ['ROUGE-1', 'ROUGE-2', 'ROUGE-L', 'BLEU', 'BERTScore (F1 Médio)', 'F1-Score (Token Médio)'],
    'Valor': [
        rouge_scores['rouge1'],
        rouge_scores['rouge2'],
        rouge_scores['rougeL'],
        bleu_scores['bleu'],
        avg_bertscore,
        avg_f1,
    ]
}
df_metrics_general = pd.DataFrame(metrics_data)
print("\nMétricas Gerais:\n")
display(df_metrics_general.round(4))

# Adicionar BERTScore F1 por pergunta ao DataFrame de resultados
df_results['bertscore_f1'] = bertscore_scores['f1']
df_results['bertscore_precision'] = bertscore_scores['precision']
df_results['bertscore_recall'] = bertscore_scores['recall']


# Exibir métricas por pergunta
print("\nMétricas por Pergunta:\n")
display(df_results[['pergunta', 'f1_score', 'bertscore_f1', 'bertscore_precision', 'bertscore_recall']].round(4))


MÉTRICAS DETALHADAS E POR PERGUNTA:


Métricas Gerais:



,Métrica,Valor
0,ROUGE-1,0.6416
1,ROUGE-2,0.4720
2,ROUGE-L,0.5351
3,BLEU,0.2528
4,BERTScore (F1 Médio),0.9223
5,F1-Score (Token Médio),0.5119



Métricas por Pergunta:



,pergunta,f1_score,bertscore_f1,bertscore_precision,bertscore_recall
0,Quando o curso de Tecnologia em Análise e Dese...,0.7200,0.9359,0.8892,0.9878
1,Quais são as modalidades aceitas para a aprese...,0.3243,0.8911,0.8448,0.9427
2,Qual foi o montante total de investimento que ...,0.4242,0.9331,0.8963,0.9730
3,Qual o pré-requisito para cursar Programação p...,0.5789,0.9292,0.8867,0.9760


from matplotlib import pyplot as plt
_df_11['f1_score'].plot(kind='hist', bins=20, title='f1_score')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_24['pergunta'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_24, x='bertscore_f1', y='pergunta', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_25['pergunta'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_25, x='bertscore_precision', y='pergunta', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_26['pergunta'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_26, x='bertscore_recall', y='pergunta', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

### 7.2 Análise Detalhada (Opcional)

In [38]:
# Análise detalhada de cada resposta
print("\nANÁLISE DETALHADA DAS RESPOSTAS:\n")

for idx, row in df_results.iterrows():
    print(f"\n{'='*80}")
    print(f"PERGUNTA {idx+1}:")
    print(f"  {row['pergunta']}")
    print(f"\nRESPOSTA ESPERADA:")
    print(f"  {row['resposta_esperada']}")
    print(f"\nRESPOSTA GERADA:")
    print(f"  {row['resposta_gerada']}")
    print(f"\nMÉTRICAS:")
    print(f"  F1-Score: {row['f1_score']:.4f}")
    print(f"  BERTScore (F1): {row['bertscore_f1']:.4f}")
    print(f"  BERTScore (Precision): {row['bertscore_precision']:.4f}")
    print(f"  BERTScore (Recall): {row['bertscore_recall']:.4f}")
    print(f"  ROUGE-1: {rouge_scores['rouge1']:.4f}")
    print(f"  ROUGE-2: {rouge_scores['rouge2']:.4f}")
    print(f"  ROUGE-L: {rouge_scores['rougeL']:.4f}")
    print(f"  BLEU: {bleu_scores['bleu']:.4f}")
    print("="*80)


ANÁLISE DETALHADA DAS RESPOSTAS:


PERGUNTA 1:
  Quando o curso de Tecnologia em Análise e Desenvolvimento de Sistemas foi implantado no Campus de Picos?

RESPOSTA ESPERADA:
  O curso foi implantado no Campus de Picos em 2013.

RESPOSTA GERADA:
  Em 2013, no Campus de Picos.

MÉTRICAS:
  F1-Score: 0.5000
  BERTScore (F1): 0.9399
  BERTScore (Precision): 0.9632
  BERTScore (Recall): 0.9177
  ROUGE-1: 0.6416
  ROUGE-2: 0.4720
  ROUGE-L: 0.5351
  BLEU: 0.2818

PERGUNTA 2:
  Quais são as modalidades aceitas para a apresentação do TCC?

RESPOSTA ESPERADA:
  O TCC pode ser apresentado em formato de Monografia, Artigo Científico, Relatório Técnico de Software (RTS) ou Relatório Técnico de Trabalho/Estágio (RTT).

RESPOSTA GERADA:
  De acordo com o contexto fornecido, as modalidades aceitas para a apresentação do TCC são:

1. Monografia;
2. Artigo Científico completo (com resultados);
3. Relatório Técnico de Software (RTS); e
4. Relatório Técnico de Trabalho/Estágio (RTT).

MÉTRICAS:
  F1-Sco

---
## 8. Salvamento dos Resultados

In [39]:
# Salvar resultados em CSV
output_filename = "resultados_rag_llama3.csv"
df_results.to_csv(output_filename, index=False)
print(f"✓ Resultados salvos em '{output_filename}'")

✓ Resultados salvos em 'resultados_rag_llama3.csv'


In [ ]:
# Salvar resumo das métricas
metrics_summary = {
    'ROUGE-1': rouge_scores['rouge1'],
    'ROUGE-2': rouge_scores['rouge2'],
    'ROUGE-L': rouge_scores['rougeL'],
    'BLEU': bleu_scores['bleu'],
    'BERTScore_F1': avg_bertscore,
    'BERTScore_Precision': sum(bertscore_scores['precision'])/len(bertscore_scores['precision']),
    'BERTScore_Recall': sum(bertscore_scores['recall'])/len(bertscore_scores['recall']),
    'F1_Token': avg_f1,
    'Total_Perguntas': len(df_results)
}

df_summary = pd.DataFrame([metrics_summary])
summary_filename = "metricas_resumo.csv"
df_summary.to_csv(summary_filename, index=False)
print(f"✓ Resumo das métricas salvo em '{summary_filename}'")

---
## 9. Teste Rápido (Pergunta Única)
**Use esta célula para testar uma pergunta específica sem avaliar métricas**

In [46]:
# Teste rápido com uma pergunta
pergunta_teste = "Qual o pré-requisito para cursar Programação para Dispositivos Móveis?"

resposta, contexto = ask(
    pergunta_teste,
    rag_chain,
    retriever,
    cross_encoder,
    verbose=True
)


PERGUNTA: Qual o pré-requisito para cursar Programação para Dispositivos Móveis?
✓ Recuperados 10 documentos

[DEBUG] Top 4 documentos após re-ranking:
  #1 (score: 2.3419): complementar: 1. WASLAWICK, Raul Sidnei. Introdução a algoritmos e programação c...
  #2 (score: -2.4429): Pré-requisito: Não possui Competência e Habilidades: Apresentação de temas atuai...
  #3 (score: -3.9853): desenvolvimento para a Internet no desenvolvimento de softwares Dinâmicos para a...
  #4 (score: -5.0021): Tulio. Engenharia de Software Moderna: Princípios e Práticas para Desenvolviment...

[DEBUG] Primeiros 400 caracteres do contexto:
--------------------------------------------------------------------------------
[Trecho 1]: complementar: 1. WASLAWICK, Raul Sidnei. Introdução a algoritmos e programação com Python: uma abordagem dirigida por testes. Rio de Janeiro: Elsevier, 2018. 2. PRESSMAN, Roger S.; MAXIM, Bruce R. Engenharia de software: uma abordagem profissional. 8. ed. São Paulo: McGraw-Hill, 

/usr/local/lib/python3.12/dist-packages/llama_cpp/llama.py:1242: RuntimeWarning: Detected duplicate leading "<|begin_of_text|>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


O pré-requisito para cursar Programação para Dispositivos Móveis é a disciplina "Programação Orientada a Objetos" (Competência e Habilidades: Compreender conhecimentos teóricos e práticos, acerca do domínio da programação para dispositivos móveis).
RESPOSTA: O pré-requisito para cursar Programação para Dispositivos Móveis é a disciplina "Programação Orientada a Objetos" (Competência e Habilidades: Compreender conhecimentos teóricos e práticos, acerca do domínio da programação para dispositivos móveis).


---
## 10. Limpeza de Memória (Opcional)
**Execute se precisar liberar recursos da GPU**

In [7]:
def clear_gpu_memory(*args):
    """Libera a memória da GPU."""
    print("\n--- Limpando a memória da GPU ---")
    for arg in args:
        try:
            del arg
        except NameError:
            pass
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        print("✓ Cache da GPU limpo.")

# Descomente para limpar memória
# clear_gpu_memory(llm, embeddings, vectorstore, cross_encoder)

---
## 11. Executar Múltiplas Rodadas de Avaliação
**Execute esta seção para rodar o dataset N vezes e obter estatísticas agregadas**

In [40]:
# Configurar número de rodadas
NUM_RODADAS = 5  # Altere para quantas rodadas desejar

print(f"Configurado para executar {NUM_RODADAS} rodadas de avaliação")

Configurado para executar 5 rodadas de avaliação


In [41]:
# Executar múltiplas rodadas
all_rounds_results = []
all_rounds_metrics = []

for rodada in range(1, NUM_RODADAS + 1):
    print(f"\n\n{'#'*80}")
    print(f"RODADA {rodada}/{NUM_RODADAS}")
    print(f"{'#'*80}\n")

    rodada_results = []

    for idx, item in enumerate(evaluation_dataset, 1):
        print(f"\n[Rodada {rodada}] Pergunta {idx}/{len(evaluation_dataset)}")

        generated_answer, retrieved_context = ask(
            item["pergunta"],
            rag_chain,
            retriever,
            cross_encoder,
            verbose=False  # Menos verboso em múltiplas rodadas
        )

        rodada_results.append({
            "rodada": rodada,
            "pergunta": item["pergunta"],
            "resposta_esperada": item["resposta_esperada"],
            "resposta_gerada": generated_answer,
            "contexto_recuperado": retrieved_context
        })

    # Calcular métricas para esta rodada
    gen_ans = [r["resposta_gerada"] for r in rodada_results]
    exp_ans = [r["resposta_esperada"] for r in rodada_results]

    rouge = rouge_metric.compute(predictions=gen_ans, references=exp_ans)
    bleu = bleu_metric.compute(predictions=gen_ans, references=exp_ans)
    bertscore = bertscore_metric.compute(
        predictions=gen_ans, references=exp_ans,
        lang='pt', model_type='distilbert-base-multilingual-cased'
    )

    f1_round = [calculate_f1_token(p, r) for p, r in zip(gen_ans, exp_ans)]

    rodada_metrics = {
        'rodada': rodada,
        'ROUGE-1': rouge['rouge1'],
        'ROUGE-2': rouge['rouge2'],
        'ROUGE-L': rouge['rougeL'],
        'BLEU': bleu['bleu'],
        'BERTScore_F1': sum(bertscore['f1']) / len(bertscore['f1']),
        'F1_Token': sum(f1_round) / len(f1_round),
    }

    all_rounds_results.extend(rodada_results)
    all_rounds_metrics.append(rodada_metrics)

    print(f"\n✓ Rodada {rodada} concluída!")
    print(f"  F1-Token: {rodada_metrics['F1_Token']:.4f}")

print(f"\n\n{'='*80}")
print(f"✓ TODAS AS {NUM_RODADAS} RODADAS CONCLUÍDAS!")
print("="*80)



################################################################################
RODADA 1/5
################################################################################


[Rodada 1] Pergunta 1/3

[DEBUG] Top 4 documentos após re-ranking:
  #1 (score: 7.1133): denominação: Análise e Desenvolvimento de Sistemas. Em 2002, foi autorizada a cr...
  #2 (score: 4.9391): O Instituto Federal do Piauí possui atualmente 20 campi distribuídos do norte ao...
  #3 (score: 4.6784): MINISTÉRIO DA EDUCAÇÃO Secretaria de Educação Profissional e Tecnológica Institu...
  #4 (score: 3.6548): de Redação do Vestibular/Exame Nacional do Ensino Médio (ENEM) em um dos últimos...


/usr/local/lib/python3.12/dist-packages/llama_cpp/llama.py:1242: RuntimeWarning: Detected duplicate leading "<|begin_of_text|>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


O curso de Tecnologia em Análise e Desenvolvimento de Sistemas foi implantado no Campus de Picos, em 2013.
[Rodada 1] Pergunta 2/3

[DEBUG] Top 4 documentos após re-ranking:
  #1 (score: 4.3135): entre o primeiro e o último período do curso; - Trabalho de Conclusão de Curso (...
  #2 (score: 4.0929): de Projeto Carga horária: 40h/a Módulo: IV Código(s) do Pré-requisito 17 a 23 Co...
  #3 (score: 4.0589): estará sujeito a uma arguição aberta aos membros da banca examinadora e à platei...
  #4 (score: 3.2253): de elaboração de trabalhos acadêmicos do IFPI. Os artigos científicos completos ...


/usr/local/lib/python3.12/dist-packages/llama_cpp/llama.py:1242: RuntimeWarning: Detected duplicate leading "<|begin_of_text|>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


De acordo com o trecho 4, as modalidades aceitas para a apresentação do TCC são:

- Monografia
- Artigo Científico completo (com resultados) publicado em eventos regionais, nacionais e internacionais ou em periódicos pertencentes à Lista de Periódicos classificados no Qualis Capes ou na Revista Somma do IFPI.

Além disso, os alunos também podem desenvolver como projeto da disciplina de TCC um Relatório Técnico de Software (RTS) ou um Relatório Técnico de Trabalho/Estágio (RTT).
[Rodada 1] Pergunta 3/3

[DEBUG] Top 4 documentos após re-ranking:
  #1 (score: 4.8680): serviços, hardware e também as exportações do segmento. Em 2021, o PIB nacional ...
  #2 (score: 1.2843): da manutenção da modalidade home-office que ainda foi necessária. Para poder con...
  #3 (score: 1.1567): % podem ser classificadas como micro e pequenas empresas, segundo análise realiz...
  #4 (score: 0.4441): que atuem na forma de consultoria em Sistemas de Informação, avaliando e selecio...


...

Em 2013, no Campus de Picos.
[Rodada 5] Pergunta 2/3

[DEBUG] Top 4 documentos após re-ranking:
  #1 (score: 4.3135): entre o primeiro e o último período do curso; - Trabalho de Conclusão de Curso (...
  #2 (score: 4.0929): de Projeto Carga horária: 40h/a Módulo: IV Código(s) do Pré-requisito 17 a 23 Co...
  #3 (score: 4.0589): estará sujeito a uma arguição aberta aos membros da banca examinadora e à platei...
  #4 (score: 3.2253): de elaboração de trabalhos acadêmicos do IFPI. Os artigos científicos completos ...


/usr/local/lib/python3.12/dist-packages/llama_cpp/llama.py:1242: RuntimeWarning: Detected duplicate leading "<|begin_of_text|>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


De acordo com o contexto fornecido, as modalidades aceitas para a apresentação do TCC são:

1. Monografia;
2. Artigo Científico completo (com resultados);
3. Relatório Técnico de Software (RTS); e
4. Relatório Técnico de Trabalho/Estágio (RTT).
[Rodada 5] Pergunta 3/3

[DEBUG] Top 4 documentos após re-ranking:
  #1 (score: 4.8680): serviços, hardware e também as exportações do segmento. Em 2021, o PIB nacional ...
  #2 (score: 1.2843): da manutenção da modalidade home-office que ainda foi necessária. Para poder con...
  #3 (score: 1.1567): % podem ser classificadas como micro e pequenas empresas, segundo análise realiz...
  #4 (score: 0.4441): que atuem na forma de consultoria em Sistemas de Informação, avaliando e selecio...


/usr/local/lib/python3.12/dist-packages/llama_cpp/llama.py:1242: RuntimeWarning: Detected duplicate leading "<|begin_of_text|>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


Em 2021, o setor de TI no Brasil atingiu um investimento total de R$238,2 bilhões (US$46,2 bilhões), englobando os mercados de software, serviços, hardware e exportações.
✓ Rodada 5 concluída!
  F1-Token: 0.3908


✓ TODAS AS 5 RODADAS CONCLUÍDAS!


In [44]:
# Análise agregada das rodadas
df_all_rounds = pd.DataFrame(all_rounds_results)
df_metrics_rounds = pd.DataFrame(all_rounds_metrics)

print("\nESTATÍSTICAS AGREGADAS DE TODAS AS RODADAS:\n")
# Display describe() output as a DataFrame
display(df_metrics_rounds.describe().round(4))

print("\n\nMÉDIAS FINAIS (todas as rodadas):\n")
final_averages_data = {}
for col in df_metrics_rounds.columns:
    if col != 'rodada':
        mean_val = df_metrics_rounds[col].mean()
        std_val = df_metrics_rounds[col].std()
        final_averages_data[col] = [mean_val, std_val]

df_final_averages = pd.DataFrame(final_averages_data, index=['Média', 'Desvio Padrão'])
display(df_final_averages.round(4))


# Calcular e exibir médias por rodada
print("\n\nMÉTRICAS MÉDIAS POR RODADA:\n")
# Ensure the relevant columns exist before grouping
metrics_cols = ['ROUGE-1', 'ROUGE-2', 'ROUGE-L', 'BLEU', 'BERTScore_F1', 'F1_Token']
df_metrics_rounds_grouped = df_metrics_rounds.groupby('rodada')[metrics_cols].mean()
display(df_metrics_rounds_grouped.round(4))


ESTATÍSTICAS AGREGADAS DE TODAS AS RODADAS:



,rodada,ROUGE-1,ROUGE-2,ROUGE-L,BLEU,BERTScore_F1,F1_Token
count,5.0000,5.0000,5.0000,5.0000,5.0000,5.0000,5.0000
mean,3.0000,0.6045,0.4489,0.5127,0.2483,0.9237,0.4196
std,1.5811,0.0356,0.0213,0.0309,0.0306,0.0056,0.0204
min,1.0000,0.5618,0.4307,0.4789,0.2257,0.9168,0.3908
25%,2.0000,0.5887,0.4307,0.4789,0.2260,0.9213,0.4162
50%,3.0000,0.5887,0.4390,0.5351,0.2260,0.9213,0.4162
75%,4.0000,0.6416,0.4720,0.5351,0.2818,0.9295,0.4281
max,5.0000,0.6416,0.4720,0.5358,0.2818,0.9295,0.4467




MÉDIAS FINAIS (todas as rodadas):



,ROUGE-1,ROUGE-2,ROUGE-L,BLEU,BERTScore_F1,F1_Token
Média,0.6045,0.4489,0.5127,0.2483,0.9237,0.4196
Desvio Padrão,0.0356,0.0213,0.0309,0.0306,0.0056,0.0204




MÉTRIAS MÉDIAS POR RODADA:



,ROUGE-1,ROUGE-2,ROUGE-L,BLEU,BERTScore_F1,F1_Token
rodada,,,,,,
1,0.5618,0.4390,0.5358,0.2257,0.9168,0.4467
2,0.6416,0.4720,0.5351,0.2818,0.9295,0.4281
3,0.5887,0.4307,0.4789,0.2260,0.9213,0.4162
4,0.5887,0.4307,0.4789,0.2260,0.9213,0.4162
5,0.6416,0.4720,0.5351,0.2818,0.9295,0.3908


In [45]:
# Salvar resultados de múltiplas rodadas
df_all_rounds.to_csv("resultados_multiplas_rodadas.csv", index=False)
df_metrics_rounds.to_csv("metricas_multiplas_rodadas.csv", index=False)

print("✓ Resultados de múltiplas rodadas salvos!")
print("  - resultados_multiplas_rodadas.csv")
print("  - metricas_multiplas_rodadas.csv")

✓ Resultados de múltiplas rodadas salvos!
  - resultados_multiplas_rodadas.csv
  - metricas_multiplas_rodadas.csv


---
## 📝 Instruções de Uso

### Fluxo Recomendado:

1. **Primeira Execução (Setup Completo):**
   - Execute seções 1 e 2 (Imports + Construção do Índice) - APENAS 1 VEZ
   - Execute seção 3 (Carregamento do Sistema RAG)
   - Execute seção 4 (Funções)

2. **Avaliações Subsequentes:**
   - Pule a seção 2 (índice já existe)
   - Execute seção 3 (Carregar sistema)
   - Execute seção 4 (Funções)
   - Configure dataset na seção 5
   - Execute seção 6 (Processar perguntas)
   - Execute seção 7 (Calcular métricas)

3. **Para Múltiplas Rodadas:**
   - Configure NUM_RODADAS na seção 11
   - Execute toda a seção 11 para análise estatística

4. **Teste Rápido:**
   - Use seção 9 para testar perguntas individuais sem avaliação formal

### Dicas:
- Os modelos são carregados **uma vez** e reutilizados
- Adicione quantas perguntas quiser na seção 5
- As métricas são calculadas de forma **independente** das perguntas
- Use a seção 10 se precisar liberar memória GPU entre execuções